## Classiq Implementation For The Research Paper: Symmetry enhanced variational quantum spin eigensolver

In [41]:
# Handle imports
from classiq import * # Using Classiq Version 0.52.0 Release
import numpy as np
# import scipy as sp

In [42]:
n_bits = 4
n_layers = 5

theta_x = np.pi
theta_y = np.pi
theta_z = np.pi

# theta_x2 = np.pi
# theta_y2 = np.pi
# theta_z2 = np.pi

# theta_x3 = np.pi
# theta_y3 = np.pi
# theta_z3 = np.pi
params = [theta_x,theta_y,theta_z]

def N_block(q, params, wires):
    
    RZ(np.pi / 2, q[wires[1]])

    CX(q[wires[1]], q[wires[0]])
     
    RZ(2* params[2] - np.pi/2,q[wires[0]])
    RY(np.pi / 2 - 2*params[0], q[wires[1]])
    
    CX(q[wires[0]], q[wires[1]])

    RY(2* params[1] - np.pi/2,q[wires[1]])

    CX(q[wires[1]], q[wires[0]])
    RZ(- np.pi / 2, q[wires[0]])

@qfunc
def main():
    q = QArray("q")
    allocate(n_bits, q)

    # First, do PauliX on all qubits
    for i in range(n_bits):
        X(q[i])

    # Next, add the Hadamards and CNOT
    for i in range(0, n_bits, 2):
        hadamard_transform(q[i])
        CX(q[i],q[i + 1])

    N_block(q, params, [0,1])


In [43]:
# SIM 
mod = create_model(main) ### Your Answer ###
qprog = synthesize(mod)### Your Answer ###
show(qprog)

/usr/bin/xdg-open: 882: x-www-browser: Permission denied
/usr/bin/xdg-open: 882: firefox: Permission denied
/usr/bin/xdg-open: 882: iceweasel: Permission denied
/usr/bin/xdg-open: 882: seamonkey: Permission denied
/usr/bin/xdg-open: 882: mozilla: Permission denied
/usr/bin/xdg-open: 882: epiphany: Permission denied
/usr/bin/xdg-open: 882: konqueror: Permission denied
/usr/bin/xdg-open: 882: chromium: Permission denied
/usr/bin/xdg-open: 882: chromium-browser: Permission denied
/usr/bin/xdg-open: 882: google-chrome: Permission denied
/usr/bin/xdg-open: 882: www-browser: Permission denied
/usr/bin/xdg-open: 882: links2: Permission denied
/usr/bin/xdg-open: 882: elinks: Permission denied
/usr/bin/xdg-open: 882: links: Permission denied
/usr/bin/xdg-open: 882: lynx: Permission denied
/usr/bin/xdg-open: 882: w3m: Permission denied
xdg-open: no method available for opening 'https://platform.classiq.io/circuit/02c376b1-fb4f-4ca5-b66c-39eb312bc50d?version=0.52.0'
